# Pre-processing Utah Allocation data for WaDEQA upload.
- Purpose:  To pre-process the Utah data into one master file for simple DataFrame creation and extraction

In [1]:
#Needed Libararies
import os
import re
import numpy as np
import pandas as pd
from datetime import datetime
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook

# Working Directory
workingDir = "C:/Users/rjame/Documents/WSWC Documents/MappingStatesDataToWaDE2.0/Utah/WaterAllocation/RawInputData"
os.chdir(workingDir)

from pyproj import Transformer, transform
transformer = Transformer.from_proj(26912, 4326)  # A trick to drastically optimize the Transformer of pyproj.
# Utah projection = EPSG:26912.  WGS84 projection used by WaDE 2.0 = epsg:4326.

## Point of Diversion Data

In [2]:
# Input Files
FI_PoD = "Point of Diversion/PointsOfDiversion_input.csv"
FI_WMs = "Point of Diversion/WRCHEX_WATER_MASTER.csv"
FI_Irr = "Point of Diversion/IRRIGATION_MASTER.csv"
FI_Mun = "Point of Diversion/WTRUSE_MUNICIPAL.csv"
FI_Pow = "Point of Diversion/WTRUSE_POWER.csv"

In [3]:
# Dataframe creation
dfPODin = pd.read_csv(FI_PoD, encoding = "ISO-8859-1") # Point of Diversion Input
dfWMain = pd.read_csv(FI_WMs, encoding = "ISO-8859-1") # Irrigation Input
dfIrrin = pd.read_csv(FI_Irr, encoding = "ISO-8859-1") # Irrigation Input
dfMunin = pd.read_csv(FI_Mun, encoding = "ISO-8859-1") # Municiplal Input
dfPowin = pd.read_csv(FI_Pow, encoding = "ISO-8859-1") # Power Inpu
dfPOD = pd.DataFrame() # Output

C:\Users\rjame\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\rjame\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (2,4,5,10,11,12,13,15,17,18,19,20,21,22,24,25,26,27,29,31,32,33,34,35,36,37,38,39,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,59,77,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,111) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\rjame\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (1,2,7,9,10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
# Merging dataframes into one, using left-join.
dfPOD = pd.merge(dfPODin, dfWMain[['WRNUM', 'DATE_FILED', 'DATE_TERMINATED', 'IRRIGATION_DEPLETION']], on='WRNUM', how='left')
dfPOD = pd.merge(dfPOD, dfIrrin[['WRNUM', 'USE_END_DATE', 'USE_BEG_DATE']], on='WRNUM', how='left')
dfPOD = pd.merge(dfPOD, dfMunin[['WRNUM', 'MUNICIPALITY']], on='WRNUM', how='left')
dfPOD = pd.merge(dfPOD, dfPowin[['WRNUM', 'POWER_CAPACITY']], on='WRNUM', how='left')
print(len(dfPOD))
dfPOD.head(3)

481411


,ï»¿OID_,OBJECTID,WRNUM,CHEXNUM,TYPE,SUMMARY_ST,STATUS,PRIORITY,USES,CFS,ACFT,LOCATION,WIN,OWNER,SOURCE,WebLink,Latitude,Longitude,DATE_FILED,DATE_TERMINATED,IRRIGATION_DEPLETION,USE_END_DATE,USE_BEG_DATE,MUNICIPALITY,POWER_CAPACITY
0,0,670852165,0005008P00,NaN,Underground,A,APPLAPP,NaN,NaN,0.0,0.0,S100 E1650 NW 20 25S 23E SL,22558,CASTLE VALLEY TOWN OF,Non-Production Well: Test,https://www.waterrights.utah.gov/search/?q=000...,38.622795,-109.401786,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,670852166,0016004M00,NaN,Underground,A,APPLAPP,NaN,NaN,0.0,0.0,N777 W2122 SE 32 7S 9W SL,22195,SCOTT REED,Non-Production Well: Unknown,https://www.waterrights.utah.gov/search/?q=001...,40.168036,-112.882368,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,670852167,0031008M00,NaN,Abandonded Well,A,APPLAPP,NaN,NaN,0.0,0.0,N1042 W1819 SE 07 4N 1W SL,22743,HILL AFB,Non-Production Well: Unknown,https://www.waterrights.utah.gov/search/?q=003...,41.091886,-111.994501,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Assign PODorPOUSite value
dfPOD['in_PODorPOUSite'] = "POD"

In [6]:
dfPOD

,ï»¿OID_,OBJECTID,WRNUM,CHEXNUM,TYPE,SUMMARY_ST,STATUS,PRIORITY,USES,CFS,ACFT,LOCATION,WIN,OWNER,SOURCE,WebLink,Latitude,Longitude,DATE_FILED,DATE_TERMINATED,IRRIGATION_DEPLETION,USE_END_DATE,USE_BEG_DATE,MUNICIPALITY,POWER_CAPACITY,in_PODorPOUSite
0,0,670852165,0005008P00,NaN,Underground,A,APPLAPP,NaN,NaN,0.0,0.0,S100 E1650 NW 20 25S 23E SL,22558,CASTLE VALLEY TOWN OF,Non-Production Well: Test,https://www.waterrights.utah.gov/search/?q=000...,38.622795,-109.401786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POD
1,1,670852166,0016004M00,NaN,Underground,A,APPLAPP,NaN,NaN,0.0,0.0,N777 W2122 SE 32 7S 9W SL,22195,SCOTT REED,Non-Production Well: Unknown,https://www.waterrights.utah.gov/search/?q=001...,40.168036,-112.882368,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POD
2,2,670852167,0031008M00,NaN,Abandonded Well,A,APPLAPP,NaN,NaN,0.0,0.0,N1042 W1819 SE 07 4N 1W SL,22743,HILL AFB,Non-Production Well: Unknown,https://www.waterrights.utah.gov/search/?q=003...,41.091886,-111.994501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POD
3,3,670852168,0055004M00,NaN,Underground,A,APPLAPP,NaN,NaN,0.0,0.0,S60 W600 NE 19 6S 3E SL,22601,QUESTAR,Non-Production Well: Unknown,https://www.waterrights.utah.gov/search/?q=005...,40.290145,-111.640689,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POD
4,4,670852169,0063002M00,NaN,Underground,A,APPLAPP,NaN,NaN,0.0,0.0,N2350 E1400 SE 29 24S 3W SL,22504,SECOR INTERNATIONAL INC.,Non-Production Well: Unknown,https://www.waterrights.utah.gov/search/?q=006...,38.683143,-112.139892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481406,364024,671216189,35-6504,E994,Underground,T,APPLWD,19760426.0,D,0.0,1.0,N700 W2100 SE 07 3N 6E SL,0,KENT E. SPENCER,Echo Reservoir,https://www.waterrights.utah.gov/search/?q=E994,41.002805,-111.311075,19760426,19930416,0,NaN,NaN,NaN,NaN,POD
481407,364025,671216190,35-6505,E995,Underground,A,APPLAPP,19760427.0,DIS,0.0,1.0,S1540 E210 N4 17 1N 5E SL,0,DIXIE LEE PECK REVOCABLE LIVING TRUST,Wanship Reservoir,https://www.waterrights.utah.gov/search/?q=E995,40.823260,-111.404884,19760427.0,NaN,0.29057,NaN,NaN,NaN,NaN,POD
481408,364026,671216191,35-6506,E997,Underground,A,APPLAPP,19760506.0,DIS,0.0,2.0,N850 E25 S4 09 1S 4E SL,12424,DAN A. PAULSON,Wanship Reservoir,https://www.waterrights.utah.gov/search/?q=E997,40.743706,-111.501450,19760506.0,NaN,0.64536,NaN,NaN,NaN,NaN,POD
481409,364027,671216192,35-6506,E997,Underground,A,APPLAPP,19760506.0,DIS,0.0,2.0,N1100 E50 S4 09 1S 4E SL,12425,CHARLES H. SUZANNE WAGNER,Wanship Reservoir,https://www.waterrights.utah.gov/search/?q=E997,40.744393,-111.501365,19760506.0,NaN,0.64536,NaN,NaN,NaN,NaN,POD


## Place of Use Data

In [7]:
# Input Files
FI_POU = "Place of Use/Utah_Place_of_Use_input.csv"

In [8]:
# Dataframe creation
dfPOU = pd.DataFrame() # Output
dfPOUin = pd.read_csv(FI_POU, encoding = "ISO-8859-1") # Place of Use Input
print(len(dfPOUin))
dfPOUin

104847


C:\Users\rjame\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ï»¿OID_,OBJECTID,GROUP_NO,WRNUMS,ACRES,CROP,METHOD,Shape__Are,Shape__Len,Latitude,Longitude
0,0,6975034,0,NaN,22.58,NaN,NaN,91398.237061,1622.371960,38.600056,-111.680779
1,1,6975035,0,NaN,2.10,NaN,NaN,8493.517700,470.191466,38.512496,-111.680026
2,2,6975036,0,NaN,45.87,NaN,NaN,185629.275146,2612.465082,38.508480,-111.681824
3,3,6975037,0,"95-640, 95-641,",0.21,NaN,NaN,838.443359,133.878365,38.507043,-111.675094
4,4,6975038,0,NaN,55.14,NaN,NaN,223149.898926,4636.586285,38.501354,-111.679579
...,...,...,...,...,...,...,...,...,...,...,...
104842,104842,7094252,0,"35-12748,",0.00,NaN,NaN,108.368286,79.837466,41.252370,-111.841450
104843,104843,7094253,0,"35-12748,",0.00,NaN,NaN,108.368286,79.837466,41.252370,-111.841450
104844,104844,7094254,0,"35-12748,",0.00,NaN,NaN,108.368286,79.837466,41.252370,-111.841450
104845,104845,7094255,0,"35-12748,",0.00,NaN,NaN,108.368286,79.837466,41.252370,-111.841450


In [9]:
# Remove empty WRNUMS rows, can't match those to anything.
def emptyWRNUMS(val):
    val = str(val).strip()
    val = val.rstrip(",")  # strip trailing commas
    return val

dfPOUin['WRNUMS'] = dfPOUin.apply(lambda row: emptyWRNUMS(row['WRNUMS']), axis=1)
dfPOUin = dfPOUin[dfPOUin['WRNUMS'] != '']
dfPOUin = dfPOUin.reset_index(drop=True)

In [10]:
# Need to split out WRNUMS into their own row
# The explode() method explodes lists into separate rows.
dfPOUin = dfPOUin.assign(WRNUMS=dfPOUin['WRNUMS'].str.split(',')).explode('WRNUMS')
dfPOUin = dfPOUin.rename({'WRNUMS': 'WRNUM'}, axis=1)
dfPOUin

,ï»¿OID_,OBJECTID,GROUP_NO,WRNUM,ACRES,CROP,METHOD,Shape__Are,Shape__Len,Latitude,Longitude
0,0,6975034,0,nan,22.58,NaN,NaN,91398.237061,1622.371960,38.600056,-111.680779
1,1,6975035,0,nan,2.10,NaN,NaN,8493.517700,470.191466,38.512496,-111.680026
2,2,6975036,0,nan,45.87,NaN,NaN,185629.275146,2612.465082,38.508480,-111.681824
3,3,6975037,0,95-640,0.21,NaN,NaN,838.443359,133.878365,38.507043,-111.675094
3,3,6975037,0,95-641,0.21,NaN,NaN,838.443359,133.878365,38.507043,-111.675094
...,...,...,...,...,...,...,...,...,...,...,...
104842,104842,7094252,0,35-12748,0.00,NaN,NaN,108.368286,79.837466,41.252370,-111.841450
104843,104843,7094253,0,35-12748,0.00,NaN,NaN,108.368286,79.837466,41.252370,-111.841450
104844,104844,7094254,0,35-12748,0.00,NaN,NaN,108.368286,79.837466,41.252370,-111.841450
104845,104845,7094255,0,35-12748,0.00,NaN,NaN,108.368286,79.837466,41.252370,-111.841450


In [11]:
# Merging dataframes into one, using left-join.
dfPOU = pd.merge(dfPOUin, dfWMain[['WRNUM', 'DATE_FILED', 'DATE_TERMINATED', 'IRRIGATION_DEPLETION']], on='WRNUM', how='left')
dfPOU = pd.merge(dfPOU, dfIrrin[['WRNUM', 'USE_END_DATE', 'USE_BEG_DATE']], on='WRNUM', how='left')
dfPOU = pd.merge(dfPOU, dfMunin[['WRNUM', 'MUNICIPALITY']], on='WRNUM', how='left')
dfPOU = pd.merge(dfPOU, dfPowin[['WRNUM', 'POWER_CAPACITY']], on='WRNUM', how='left')
print(len(dfPOU))
dfPOU.head(3)

564852


,ï»¿OID_,OBJECTID,GROUP_NO,WRNUM,ACRES,CROP,METHOD,Shape__Are,Shape__Len,Latitude,Longitude,DATE_FILED,DATE_TERMINATED,IRRIGATION_DEPLETION,USE_END_DATE,USE_BEG_DATE,MUNICIPALITY,POWER_CAPACITY
0,0,6975034,0,nan,22.58,NaN,NaN,91398.237061,1622.371960,38.600056,-111.680779,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,6975035,0,nan,2.10,NaN,NaN,8493.517700,470.191466,38.512496,-111.680026,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,6975036,0,nan,45.87,NaN,NaN,185629.275146,2612.465082,38.508480,-111.681824,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Assign PODorPOUSite value
dfPOU['in_PODorPOUSite'] = "POU"

In [13]:
dfPOU

,ï»¿OID_,OBJECTID,GROUP_NO,WRNUM,ACRES,CROP,METHOD,Shape__Are,Shape__Len,Latitude,Longitude,DATE_FILED,DATE_TERMINATED,IRRIGATION_DEPLETION,USE_END_DATE,USE_BEG_DATE,MUNICIPALITY,POWER_CAPACITY,in_PODorPOUSite
0,0,6975034,0,nan,22.58,NaN,NaN,91398.237061,1622.371960,38.600056,-111.680779,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POU
1,1,6975035,0,nan,2.10,NaN,NaN,8493.517700,470.191466,38.512496,-111.680026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POU
2,2,6975036,0,nan,45.87,NaN,NaN,185629.275146,2612.465082,38.508480,-111.681824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POU
3,3,6975037,0,95-640,0.21,NaN,NaN,838.443359,133.878365,38.507043,-111.675094,19670301.0,NaN,232.018,1031,501.0,NaN,NaN,POU
4,3,6975037,0,95-641,0.21,NaN,NaN,838.443359,133.878365,38.507043,-111.675094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564847,104842,7094252,0,35-12748,0.00,NaN,NaN,108.368286,79.837466,41.252370,-111.841450,20130528.0,NaN,0,NaN,NaN,NaN,NaN,POU
564848,104843,7094253,0,35-12748,0.00,NaN,NaN,108.368286,79.837466,41.252370,-111.841450,20130528.0,NaN,0,NaN,NaN,NaN,NaN,POU
564849,104844,7094254,0,35-12748,0.00,NaN,NaN,108.368286,79.837466,41.252370,-111.841450,20130528.0,NaN,0,NaN,NaN,NaN,NaN,POU
564850,104845,7094255,0,35-12748,0.00,NaN,NaN,108.368286,79.837466,41.252370,-111.841450,20130528.0,NaN,0,NaN,NaN,NaN,NaN,POU


## Concatenate POD and POU Data

In [14]:
# Concatenate
frames = [dfPOD, dfPOU]
dfout = pd.concat(frames)

#Removing all NaN Values and replacing with blank
dfout = dfout.replace(np.nan, "", regex=True)

print(len(dfout))
dfout

1046263


,ï»¿OID_,OBJECTID,WRNUM,CHEXNUM,TYPE,SUMMARY_ST,STATUS,PRIORITY,USES,CFS,ACFT,LOCATION,WIN,OWNER,SOURCE,WebLink,Latitude,Longitude,DATE_FILED,DATE_TERMINATED,IRRIGATION_DEPLETION,USE_END_DATE,USE_BEG_DATE,MUNICIPALITY,POWER_CAPACITY,in_PODorPOUSite,GROUP_NO,ACRES,CROP,METHOD,Shape__Are,Shape__Len
0,0,670852165,0005008P00,,Underground,A,APPLAPP,,,0.0,0.0,S100 E1650 NW 20 25S 23E SL,22558.0,CASTLE VALLEY TOWN OF,Non-Production Well: Test,https://www.waterrights.utah.gov/search/?q=000...,38.622795,-109.401786,,,,,,,,POD,,,,,,
1,1,670852166,0016004M00,,Underground,A,APPLAPP,,,0.0,0.0,N777 W2122 SE 32 7S 9W SL,22195.0,SCOTT REED,Non-Production Well: Unknown,https://www.waterrights.utah.gov/search/?q=001...,40.168036,-112.882368,,,,,,,,POD,,,,,,
2,2,670852167,0031008M00,,Abandonded Well,A,APPLAPP,,,0.0,0.0,N1042 W1819 SE 07 4N 1W SL,22743.0,HILL AFB,Non-Production Well: Unknown,https://www.waterrights.utah.gov/search/?q=003...,41.091886,-111.994501,,,,,,,,POD,,,,,,
3,3,670852168,0055004M00,,Underground,A,APPLAPP,,,0.0,0.0,S60 W600 NE 19 6S 3E SL,22601.0,QUESTAR,Non-Production Well: Unknown,https://www.waterrights.utah.gov/search/?q=005...,40.290145,-111.640689,,,,,,,,POD,,,,,,
4,4,670852169,0063002M00,,Underground,A,APPLAPP,,,0.0,0.0,N2350 E1400 SE 29 24S 3W SL,22504.0,SECOR INTERNATIONAL INC.,Non-Production Well: Unknown,https://www.waterrights.utah.gov/search/?q=006...,38.683143,-112.139892,,,,,,,,POD,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564847,104842,7094252,35-12748,,,,,,,,,,,,,,41.252370,-111.841450,20130528.0,,0,,,,,POU,0.0,0.0,,,108.368286,79.837466
564848,104843,7094253,35-12748,,,,,,,,,,,,,,41.252370,-111.841450,20130528.0,,0,,,,,POU,0.0,0.0,,,108.368286,79.837466
564849,104844,7094254,35-12748,,,,,,,,,,,,,,41.252370,-111.841450,20130528.0,,0,,,,,POU,0.0,0.0,,,108.368286,79.837466
564850,104845,7094255,35-12748,,,,,,,,,,,,,,41.252370,-111.841450,20130528.0,,0,,,,,POU,0.0,0.0,,,108.368286,79.837466


In [15]:
# Making Sure datatype of Long, Lat, Wrex, Irrigation are Float
dfout['CFS'] = pd.to_numeric(dfout['CFS'], errors='coerce')
dfout['ACFT'] = pd.to_numeric(dfout['ACFT'], errors='coerce')
dfout['IRRIGATION_DEPLETION'] = pd.to_numeric(dfout['IRRIGATION_DEPLETION'], errors='coerce')

In [16]:
# Changing datatype of used date fields. 
dfout['PRIORITY'] = pd.to_datetime(dfout['PRIORITY'], errors = 'coerce')
dfout['PRIORITY'] = pd.to_datetime(dfout["PRIORITY"].dt.strftime('%m/%d/%Y'))

dfout['DATE_FILED'] = pd.to_datetime(dfout['DATE_FILED'], errors = 'coerce')
dfout['DATE_FILED'] = pd.to_datetime(dfout["DATE_FILED"].dt.strftime('%m/%d/%Y'))

dfout['DATE_TERMINATED'] = pd.to_datetime(dfout['DATE_TERMINATED'], errors = 'coerce')
dfout['DATE_TERMINATED'] = pd.to_datetime(dfout["DATE_TERMINATED"].dt.strftime('%m/%d/%Y'))

In [17]:
# Creating WaterSourceTypeCV

WaterSourceTypeCVDictionary={
"Underground" : "Groundwater",
"Abandonded Well" : "Groundwater",
"Point to Point" : "Surface Water",
"Surface" : "Surface Water",
"Return" : "Surface Water",
"Drain" : "Surface Water",
"Spring" : "Surface Water",
"Rediversion" : "Surface Water"}
def CreateWaterSourceTypeCV(val):
    if val == '' or pd.isnull(val):
        outString = "Unspecified"
    else:
        val = val.strip()
        try:
            outString = WaterSourceTypeCVDictionary[val]
        except:
            outString = "Unspecified"
    return outString

dfout['in_WaterSourceTypeCV'] = dfout.apply(lambda row: CreateWaterSourceTypeCV(row['TYPE']), axis=1)
dfout

,ï»¿OID_,OBJECTID,WRNUM,CHEXNUM,TYPE,SUMMARY_ST,STATUS,PRIORITY,USES,CFS,ACFT,LOCATION,WIN,OWNER,SOURCE,WebLink,Latitude,Longitude,DATE_FILED,DATE_TERMINATED,IRRIGATION_DEPLETION,USE_END_DATE,USE_BEG_DATE,MUNICIPALITY,POWER_CAPACITY,in_PODorPOUSite,GROUP_NO,ACRES,CROP,METHOD,Shape__Are,Shape__Len,in_WaterSourceTypeCV
0,0,670852165,0005008P00,,Underground,A,APPLAPP,NaT,,0.0,0.0,S100 E1650 NW 20 25S 23E SL,22558.0,CASTLE VALLEY TOWN OF,Non-Production Well: Test,https://www.waterrights.utah.gov/search/?q=000...,38.622795,-109.401786,NaT,NaT,NaN,,,,,POD,,,,,,,Groundwater
1,1,670852166,0016004M00,,Underground,A,APPLAPP,NaT,,0.0,0.0,N777 W2122 SE 32 7S 9W SL,22195.0,SCOTT REED,Non-Production Well: Unknown,https://www.waterrights.utah.gov/search/?q=001...,40.168036,-112.882368,NaT,NaT,NaN,,,,,POD,,,,,,,Groundwater
2,2,670852167,0031008M00,,Abandonded Well,A,APPLAPP,NaT,,0.0,0.0,N1042 W1819 SE 07 4N 1W SL,22743.0,HILL AFB,Non-Production Well: Unknown,https://www.waterrights.utah.gov/search/?q=003...,41.091886,-111.994501,NaT,NaT,NaN,,,,,POD,,,,,,,Groundwater
3,3,670852168,0055004M00,,Underground,A,APPLAPP,NaT,,0.0,0.0,S60 W600 NE 19 6S 3E SL,22601.0,QUESTAR,Non-Production Well: Unknown,https://www.waterrights.utah.gov/search/?q=005...,40.290145,-111.640689,NaT,NaT,NaN,,,,,POD,,,,,,,Groundwater
4,4,670852169,0063002M00,,Underground,A,APPLAPP,NaT,,0.0,0.0,N2350 E1400 SE 29 24S 3W SL,22504.0,SECOR INTERNATIONAL INC.,Non-Production Well: Unknown,https://www.waterrights.utah.gov/search/?q=006...,38.683143,-112.139892,NaT,NaT,NaN,,,,,POD,,,,,,,Groundwater
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564847,104842,7094252,35-12748,,,,,NaT,,NaN,NaN,,,,,,41.252370,-111.841450,1970-01-01,NaT,0.0,,,,,POU,0.0,0.0,,,108.368286,79.837466,Unspecified
564848,104843,7094253,35-12748,,,,,NaT,,NaN,NaN,,,,,,41.252370,-111.841450,1970-01-01,NaT,0.0,,,,,POU,0.0,0.0,,,108.368286,79.837466,Unspecified
564849,104844,7094254,35-12748,,,,,NaT,,NaN,NaN,,,,,,41.252370,-111.841450,1970-01-01,NaT,0.0,,,,,POU,0.0,0.0,,,108.368286,79.837466,Unspecified
564850,104845,7094255,35-12748,,,,,NaT,,NaN,NaN,,,,,,41.252370,-111.841450,1970-01-01,NaT,0.0,,,,,POU,0.0,0.0,,,108.368286,79.837466,Unspecified


In [18]:
#Compiling 'AllocationTimeframeStart' & 'AllocationTimeframeEnd'
#Both can have a string format for WaDE 2.0.
    
def assignTime(colrowValue):
    if colrowValue == "" or pd.isnull(colrowValue):
        outlist = ""
    else:
        colrowValue = str(colrowValue)
        colrowValue = colrowValue.strip()
        if len(colrowValue) == 4:
            startmonth = colrowValue[0:1]
            startday = colrowValue[1:2]
            outlist = "0" + startmonth + "/" + "0" + startday
        elif len(colrowValue) == 5:
            startmonth = colrowValue[0:1]
            startday = colrowValue[1:3]
            outlist = "0" + startmonth + "/" + startday
        elif len(colrowValue) == 6:
            startmonth = colrowValue[0:2]
            startday = colrowValue[2:4]
            outlist = startmonth + "/" + startday
        else:
            outlist = ""
  
    return outlist


dfout['in_AllocationTimeframeStart'] = dfout.apply(lambda row: assignTime(row['USE_BEG_DATE']), axis=1)
dfout['in_AllocationTimeframeEnd'] = dfout.apply(lambda row: assignTime(row['USE_END_DATE']), axis=1)

In [19]:
# Assign SiteTypeCV value.
# Uses the re library, but requires for loop.
# Order that the lists are inputed into dictoinary is important, want to overide generic search with a more specific search.

# Create the Lists
canalList = ["canal", "canals"]
creekList = ["creek"]
ditchList = ["ditch"]
drainList = ["drain", "drains"]
lakeList = ["lake"]
pondList = ["pond"]
reservoirList = ["reservoir"]
riverList = ["river", "fork", "surface"]
sloughList = ["slough"]
springList = ["spring", "springs", "gulch", "seep"]
tunnelList = ["tunnel", "tunnels"]
washList = ["wash"]
wellList = ["well", "wells", "well:", "draw", "hollow"]

# Making the dictionary
listDictionary = {}
listDictionary["Canal"] = canalList
listDictionary["Creek"] = creekList
listDictionary["Ditch"] = ditchList
listDictionary["Drain"] = drainList
listDictionary["Lake"] = lakeList
listDictionary["Pond"] = pondList
listDictionary["Reservoir"] = reservoirList
listDictionary["River"] = riverList
listDictionary["Slough"] = sloughList
listDictionary["Spring"] = springList
listDictionary["Tunnel"] = tunnelList
listDictionary["Wash"] = washList
listDictionary["Well"] = wellList

def CreateSiteTypeCV(val):
    if val == '' or pd.isnull(val):
        outString = "Unspecified"
    else:
        outString = "Unspecified" # Default
        
        # Cleaning text / simple search format
        val = val.replace(",", " ")
        val = val.replace(".", " ")
        val = val.replace(";", " ")
        val = val.replace("-", " ")
        val = val.replace("/", " ")
        val = val.replace("(", " ")
        val = val.replace(")", " ")
        val = val.lower().strip()
        val = " "+val+" "
        
        for x in listDictionary:
            labelString = x
            valueList = listDictionary[x]
            for words in valueList:
                if re.search(" "+words+ " ", val): outString = x
            
    return outString

dfout['in_SiteTypeCV'] = dfout.apply(lambda row: CreateSiteTypeCV( row['SOURCE']), axis=1)

In [20]:
# Assign LegalStatusCV value.
# Uses the re library, but requires for loop.
# Order that the lists are inputed into dictoinary is important, want to overide generic search with a more specific search.

# Create the Lists
ADECList = ["ADEC"]
ADVList = ["ADV"]
APPList = ["APP"]
CERTList = ["CERT"]
DECList = ["DEC"]
DILList = ["DIL"]
DISList = ["DIS"]
EXPList = ["EXP"]
FORFList = ["FORF"]
LAPList = ["LAP"]
NPRList = ["NPR"]
NUSEList = ["NUSE"]
PERFList = ["PERF"]
REJList = ["REJ"]
RNUMList = ["RNUM"]
STATUSList = ["STATUS"]
TEMPList = ["TEMP"]
TERMList = ["TERM"]
UGWCList = ["UGWC"]
UNAPList = ["UNAP"]
WDList = ["WD"]
WUCList = ["WUC"]


# Making the dictionary
listDictionary = {}

listDictionary["Lapsed"] = LAPList

listDictionary["Adjudication Decree"] = ADECList
listDictionary["Adverse Use Claim"] = ADVList
listDictionary["Approved"] = APPList
listDictionary["Certificated"] = CERTList
listDictionary["Decree"] = DECList
listDictionary["Diligence Claim"] = DILList
listDictionary["Disallowed"] = DISList
listDictionary["Expired"] = EXPList
listDictionary["Forfeited"] = FORFList
listDictionary["No Proof Required"] = NPRList
listDictionary["Nonuse"] = NUSEList
listDictionary["Perfected"] = PERFList
listDictionary["Rejected"] = REJList
listDictionary["Renumbered"] = RNUMList
listDictionary["Deff"] = STATUSList
listDictionary["Temp Applications"] = TEMPList
listDictionary["Terminated"] = TERMList
listDictionary["Underground Water Claim"] = UGWCList
listDictionary["Unapproved"] = UNAPList
listDictionary["Withdrawn"] = WDList
listDictionary["Water User`s Claim"] = WUCList


def CreateLegalStatus(val):
    val = str(val).strip()
    if val == "" or pd.isnull(val):
        outString = ""
    else:
        outString = ""
        for x in listDictionary:
            valueList = listDictionary[x]
            for words in valueList:
                if words in val: outString = x

    return outString

dfout['in_LegalStatus'] = dfout.apply(lambda row: CreateLegalStatus( row['STATUS']), axis=1)

## WaDE Custom Elements (due to missing state site info)

In [21]:
# Creating WaDE Custom water source native ID for easy water source identification
# ----------------------------------------------------------------------------------------------------

# Create temp WaterSourceNativeID dataframe of unique water source.
def assignWaterSourceNativeID(colrowValue):
    string1 = str(colrowValue)
    outstring = "WaDEUT_WS" + string1
    return outstring

dfWaterSourceNativeID = pd.DataFrame()
dfWaterSourceNativeID['in_WaterSourceTypeCV'] = dfout['in_WaterSourceTypeCV']
dfWaterSourceNativeID = dfWaterSourceNativeID.drop_duplicates()

dftemp = pd.DataFrame(index=dfWaterSourceNativeID.index)
dftemp["Count"] = range(1, len(dftemp.index) + 1)
dfWaterSourceNativeID['in_WaterSourceNativeID'] = dftemp.apply(lambda row: assignWaterSourceNativeID(row['Count']), axis=1)

# ----------------------------------------------------------------------------------------------------

# Retreive WaDE Custom water source native ID
def retrieveWaterSourceNativeID(A):
    if (A == '') or (pd.isnull(A)):
        outList = ''
    else:
        ml = dfWaterSourceNativeID.loc[(dfWaterSourceNativeID['in_WaterSourceTypeCV'] == A), 'in_WaterSourceNativeID']
        if not (ml.empty):  # check if the series is empty
            outList = ml.iloc[0]
        else:
            outList = ''
    return outList

dfout['in_WaterSourceNativeID'] = dfout.apply(lambda row: retrieveWaterSourceNativeID( row['in_WaterSourceTypeCV']), axis=1)

In [22]:
#Removing all NaN Values and replacing with blank
dfout = dfout.replace(np.nan, "", regex=True)

dfout

,ï»¿OID_,OBJECTID,WRNUM,CHEXNUM,TYPE,SUMMARY_ST,STATUS,PRIORITY,USES,CFS,ACFT,LOCATION,WIN,OWNER,SOURCE,WebLink,Latitude,Longitude,DATE_FILED,DATE_TERMINATED,IRRIGATION_DEPLETION,USE_END_DATE,USE_BEG_DATE,MUNICIPALITY,POWER_CAPACITY,in_PODorPOUSite,GROUP_NO,ACRES,CROP,METHOD,Shape__Are,Shape__Len,in_WaterSourceTypeCV,in_AllocationTimeframeStart,in_AllocationTimeframeEnd,in_SiteTypeCV,in_LegalStatus,in_WaterSourceNativeID
0,0,670852165,0005008P00,,Underground,A,APPLAPP,,,0.0,0.0,S100 E1650 NW 20 25S 23E SL,22558.0,CASTLE VALLEY TOWN OF,Non-Production Well: Test,https://www.waterrights.utah.gov/search/?q=000...,38.622795,-109.401786,,,,,,,,POD,,,,,,,Groundwater,,,Well,Approved,WaDEUT_WS1
1,1,670852166,0016004M00,,Underground,A,APPLAPP,,,0.0,0.0,N777 W2122 SE 32 7S 9W SL,22195.0,SCOTT REED,Non-Production Well: Unknown,https://www.waterrights.utah.gov/search/?q=001...,40.168036,-112.882368,,,,,,,,POD,,,,,,,Groundwater,,,Well,Approved,WaDEUT_WS1
2,2,670852167,0031008M00,,Abandonded Well,A,APPLAPP,,,0.0,0.0,N1042 W1819 SE 07 4N 1W SL,22743.0,HILL AFB,Non-Production Well: Unknown,https://www.waterrights.utah.gov/search/?q=003...,41.091886,-111.994501,,,,,,,,POD,,,,,,,Groundwater,,,Well,Approved,WaDEUT_WS1
3,3,670852168,0055004M00,,Underground,A,APPLAPP,,,0.0,0.0,S60 W600 NE 19 6S 3E SL,22601.0,QUESTAR,Non-Production Well: Unknown,https://www.waterrights.utah.gov/search/?q=005...,40.290145,-111.640689,,,,,,,,POD,,,,,,,Groundwater,,,Well,Approved,WaDEUT_WS1
4,4,670852169,0063002M00,,Underground,A,APPLAPP,,,0.0,0.0,N2350 E1400 SE 29 24S 3W SL,22504.0,SECOR INTERNATIONAL INC.,Non-Production Well: Unknown,https://www.waterrights.utah.gov/search/?q=006...,38.683143,-112.139892,,,,,,,,POD,,,,,,,Groundwater,,,Well,Approved,WaDEUT_WS1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564847,104842,7094252,35-12748,,,,,,,,,,,,,,41.252370,-111.841450,1970-01-01 00:00:00,,0.0,,,,,POU,0.0,0.0,,,108.368286,79.837466,Unspecified,,,Unspecified,,WaDEUT_WS3
564848,104843,7094253,35-12748,,,,,,,,,,,,,,41.252370,-111.841450,1970-01-01 00:00:00,,0.0,,,,,POU,0.0,0.0,,,108.368286,79.837466,Unspecified,,,Unspecified,,WaDEUT_WS3
564849,104844,7094254,35-12748,,,,,,,,,,,,,,41.252370,-111.841450,1970-01-01 00:00:00,,0.0,,,,,POU,0.0,0.0,,,108.368286,79.837466,Unspecified,,,Unspecified,,WaDEUT_WS3
564850,104845,7094255,35-12748,,,,,,,,,,,,,,41.252370,-111.841450,1970-01-01 00:00:00,,0.0,,,,,POU,0.0,0.0,,,108.368286,79.837466,Unspecified,,,Unspecified,,WaDEUT_WS3


In [23]:
#Exporting to Finished File
dfout.to_csv('P_UtahMaster.csv', index=False)  # The output